# Data Preprocessing

1. Data Selection
    * We decided to use [Diabetes 130-US hospitals for years 1999-2008 Data-Set](https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008# "Diabetes 130-US hospitals for years 1999-2008 Data Set") for our readmission rate predicitions. 


## Data preprocessing

1) Drop unwanted columns : patient_nbr, encounter_id, weight, payer_code, medical_specialty.

2) Drop columns 'citoglipton', 'examide' which has the same values across all the rows. So these columns won't help us in modeling

2) cleaned up race column ( replaced ? to another category and applied LabelEncoder).

3) drop rows which has invalid gender.

4) cleaned age column by removing the interval and putting the median value. 

5) assigned 3 categories to admission_type_id column

6) assigned 3 categories to admission_source_id column

7) remove any rows where the patient is expired based on discharge_disposition_id = 11

8) cleaning up the max glu serum into 3 categories

9) cleaning up the A1Cresult

10) cleaning up the diag columns


In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn import preprocessing

# read the data set.
dirPath = 'dataset_diabetes/'
df = pd.read_csv(dirPath + "diabetic_data.csv")
index_mapping =  pd.read_csv(dirPath + "IDs_mapping.csv")

dsdict = dict()
encoders = dict()
metadata = dict()
tempDict = dict()


In [2]:
#chceck the shape of the dataset
df.shape

(101766, 50)

In [3]:
#describe the data
df.describe()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,1.017660e+05,1.017660e+05,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,1.652016e+08,5.433040e+07,2.024006,3.715642,5.754437,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,0.635566,7.422607
std,1.026403e+08,3.869636e+07,1.445403,5.280166,4.064081,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.262863,1.933600
min,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,8.496119e+07,2.341322e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,1.523890e+08,4.550514e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,2.302709e+08,8.754595e+07,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


In [4]:
#describe the info of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
encounter_id                101766 non-null int64
patient_nbr                 101766 non-null int64
race                        101766 non-null object
gender                      101766 non-null object
age                         101766 non-null object
weight                      101766 non-null object
admission_type_id           101766 non-null int64
discharge_disposition_id    101766 non-null int64
admission_source_id         101766 non-null int64
time_in_hospital            101766 non-null int64
payer_code                  101766 non-null object
medical_specialty           101766 non-null object
num_lab_procedures          101766 non-null int64
num_procedures              101766 non-null int64
num_medications             101766 non-null int64
number_outpatient           101766 non-null int64
number_emergency            101766 non-null int64
number_inpatient            10176

In [5]:
# calculate all the missing values which are represented by '?'
for colums in df.columns:
    if df[colums].dtype == object:
         print(colums, df[colums][df[colums] == '?'].count())
print('gender', df['gender'][df['gender'] == 'Unknown/Invalid'].count())

race 2273
gender 0
age 0
weight 98569
payer_code 40256
medical_specialty 49949
diag_1 21
diag_2 358
diag_3 1423
max_glu_serum 0
A1Cresult 0
metformin 0
repaglinide 0
nateglinide 0
chlorpropamide 0
glimepiride 0
acetohexamide 0
glipizide 0
glyburide 0
tolbutamide 0
pioglitazone 0
rosiglitazone 0
acarbose 0
miglitol 0
troglitazone 0
tolazamide 0
examide 0
citoglipton 0
insulin 0
glyburide-metformin 0
glipizide-metformin 0
glimepiride-pioglitazone 0
metformin-rosiglitazone 0
metformin-pioglitazone 0
change 0
diabetesMed 0
readmitted 0
gender 3


In [6]:
# now we need to drop some columns which are not being helpful to us such as patient_nbr, encounter_id, weight, payer_code
# medical_specialty, citoglipton, examide
df = df.drop(['patient_nbr', 'encounter_id', 'weight', 'payer_code', 'medical_specialty', 'citoglipton', 'examide'], axis=1)

#also dropping rows where gender is invalid
df = df[df.gender != 'Unknown/Invalid']

# dropping people who expired already, since they won't be readmitting and counting those records will impact our predictions
# negatively
df = df[df.discharge_disposition_id != 11]
df = df[df.discharge_disposition_id != 19]
df = df[df.discharge_disposition_id != 20]
df = df[df.discharge_disposition_id != 21]
print('discharge_disposition_id', df['discharge_disposition_id'][df['discharge_disposition_id'] == 11].count())


discharge_disposition_id 0


In [7]:


def replaceColumn(df, col, oldval, newval, add=True):
    if (add):
        vall = dsdict.get(col)
        if (vall == None):
            dsdict.update({col:dict()})
            metadata.update({col:"manual"})
            dictNew = dsdict.get(col)
            dictNew.update({"unknown": -1})
        dictNew = dsdict.get(col)
        dictNew.update({oldval: newval})
    df[col] = df[col].replace(oldval, newval)
    return df

def replaceColumnList(df, col, listOfOldVal, newval):
    newDf = df
    for oldVal in listOfOldVal:
        newDf = replaceColumn(df, col, oldVal, newval)
    return newDf

def transformLabelEncoder(df, col):
    le = preprocessing.LabelEncoder()
    df[col] = le.fit_transform(df[col])
    filename = col+".npy"
    fullFilename = 'dataset_diabetes/' + filename
    np.save(fullFilename, le.classes_)
    insertOrUpdate(encoders, col, fullFilename)
    return df;

def insertOrUpdate(encoders, col, encodeclass):
    vall = encoders.get(col)
    if (vall == None):
        encoders.update({col:encodeclass})
        metadata.update({col:"labelencoder"})


In [8]:
# fixing age column, putting median 
# TODO maybe send a list of tuple to do mass replace in 1 shot

df = replaceColumn(df, 'age', '[0-10)', 4)
df = replaceColumn(df, 'age', '[10-20)', 14)
df = replaceColumn(df, 'age', '[20-30)', 24)
df = replaceColumn(df, 'age', '[30-40)', 34)
df = replaceColumn(df, 'age', '[40-50)', 44)
df = replaceColumn(df, 'age', '[50-60)', 54)
df = replaceColumn(df, 'age', '[60-70)', 64)
df = replaceColumn(df, 'age', '[70-80)', 74)
df = replaceColumn(df, 'age', '[80-90)', 84)
df = replaceColumn(df, 'age', '[90-100)', 94)

# fixing admission_type_id column
df = replaceColumnList(df, 'admission_type_id', [2,7], 1)
df = replaceColumnList(df, 'admission_type_id', [6,8], 5)
df = replaceColumnList(df, 'admission_type_id', [4], 3)

# fixing admission_source_id column
df = replaceColumnList(df, 'admission_source_id', [2,3], 1)
df = replaceColumnList(df, 'admission_source_id', [5,6,22], 4)
df = replaceColumnList(df, 'admission_source_id', [10,25], 7)
df = replaceColumnList(df, 'admission_source_id', [17,20], 9)
df = replaceColumnList(df, 'admission_source_id', [13,14], 11)

print('admission_type_id', df['admission_type_id'][df['admission_type_id'] == 7].count())
print('admission_source_id', df['admission_source_id'][df['admission_source_id'] == 2].count())
print('discharge_disposition_id', df['discharge_disposition_id'][df['discharge_disposition_id'] == 11].count())


admission_type_id 0
admission_source_id 0
discharge_disposition_id 0


In [9]:

# cleaning up the max glu serum
df = replaceColumnList(df, 'max_glu_serum', ['>300', '>200'], 1)
df = replaceColumnList(df, 'max_glu_serum', ['Norm'], 0)
df = replaceColumnList(df, 'max_glu_serum', ['None'], -1)

print('max_glu_serum : 1', df['max_glu_serum'][df['max_glu_serum'] == 1].count())
print('max_glu_serum : 0', df['max_glu_serum'][df['max_glu_serum'] == 0].count())

# cleaning up the A1Cresult
df = replaceColumnList(df, 'A1Cresult', ['>7', '>8'], 1)
df = replaceColumnList(df, 'A1Cresult', ['Norm'], 0)
df = replaceColumnList(df, 'A1Cresult', ['None'], -1)
print('A1Cresult : 1', df['A1Cresult'][df['A1Cresult'] == 1].count())
print('A1Cresult : 0', df['A1Cresult'][df['A1Cresult'] == 0].count())

# fixing race column and tranforming into label
df = transformLabelEncoder(df, "race")



max_glu_serum : 1 2651
max_glu_serum : 0 2573
A1Cresult : 1 11935
A1Cresult : 0 4941


In [10]:

def diagColn(df):
    colList = ['diag_1','diag_2','diag_3']
    for col in colList:
        metadata.update({col: "diag"})
        df.loc[df[col].str.contains('E'), col] = '0'
        df.loc[df[col].str.contains('V'), col] = '0'
        df = replaceColumn(df, col, '?', -1, False)
        df = replaceColumn(df, col, '0', 0, False)
        df[col] = pd.to_numeric(df[col])

        df[col] = np.where(df[col].between(1, 139, inclusive=True), 1, df[col])
        df[col] = np.where(df[col].between(140, 239, inclusive=True), 2, df[col])
        df[col] = np.where(df[col].between(240, 279, inclusive=True), 3, df[col])
        df[col] = np.where(df[col].between(280, 289, inclusive=True), 4, df[col])
        df[col] = np.where(df[col].between(290, 319, inclusive=True), 5, df[col])
        df[col] = np.where(df[col].between(320, 389, inclusive=True), 6, df[col])
        df[col] = np.where(df[col].between(390, 459, inclusive=True), 7, df[col])
        df[col] = np.where(df[col].between(460, 519, inclusive=True), 8, df[col])
        df[col] = np.where(df[col].between(520, 579, inclusive=True), 9, df[col])
        df[col] = np.where(df[col].between(580, 629, inclusive=True), 10, df[col])
        df[col] = np.where(df[col].between(630, 679, inclusive=True), 11, df[col])
        df[col] = np.where(df[col].between(680, 709, inclusive=True), 12, df[col])
        df[col] = np.where(df[col].between(710, 739, inclusive=True), 13, df[col])
        df[col] = np.where(df[col].between(740, 759, inclusive=True), 14, df[col])
        df[col] = np.where(df[col].between(760, 779, inclusive=True), 15, df[col])
        df[col] = np.where(df[col].between(780, 799, inclusive=True), 16, df[col])
        df[col] = np.where(df[col].between(800, 999, inclusive=True), 17, df[col])
    return df

# cleaning up the diag columns
df = diagColn(df)
print('diag_1 : 0', df['diag_1'][df['diag_1'] == 250.83].count())


diag_1 : 0 0


In [11]:
# using label encoder, here we are transforming categories into labels for all the medicines 
# and change column and diabetesMed Column

drugEncoder = preprocessing.LabelEncoder()

drugEncoder.fit(df['metformin'])
filename = "drug.npy"
fullFilename = dirPath + filename
insertOrUpdate(encoders, "drug", fullFilename)
np.save(fullFilename, drugEncoder.classes_)

for name,values in df.loc[:, 'metformin': 'metformin-pioglitazone'].iteritems():
    df[name] = drugEncoder.transform(values)
print("map of encoder: " + str(list(drugEncoder.classes_)))


df = transformLabelEncoder(df, 'change')
df = transformLabelEncoder(df, 'diabetesMed')
df = replaceColumnList(df, 'readmitted', ['>30','<30'], 1)
df = replaceColumnList(df, 'readmitted', ['NO'], 0)

df = transformLabelEncoder(df, 'gender')


map of encoder: ['Down', 'No', 'Steady', 'Up']


In [12]:
#Run the normalizer on the dataframe
# df_norm = normalize(df, 'time_in_hospital')
# df_norm = normalize(df_norm, 'num_lab_procedures')
# df_norm = normalize(df_norm, 'num_procedures')
# df_norm = normalize(df_norm, 'num_medications')
# df_norm = normalize(df_norm, 'number_outpatient')
# df_norm = normalize(df_norm, 'number_emergency')
# df_norm = normalize(df_norm, 'number_inpatient')
# df_norm
# df
print('readmitted', df['readmitted'][df['readmitted'] == 0].count())
print('readmitted', df['readmitted'][df['readmitted'] == 1].count())

metadataList = list()
metadataList.append(metadata)
tempDict.update({"manual": dsdict, "labelencoder": encoders})
metadataList.append(tempDict)

print(metadataList)
pickle.dump(metadataList, open(dirPath +  "metadataList.p", "wb" ) )

readmitted 53209
readmitted 46902
[{'age': 'manual', 'admission_type_id': 'manual', 'admission_source_id': 'manual', 'max_glu_serum': 'manual', 'A1Cresult': 'manual', 'race': 'labelencoder', 'diag_1': 'diag', 'diag_2': 'diag', 'diag_3': 'diag', 'drug': 'labelencoder', 'change': 'labelencoder', 'diabetesMed': 'labelencoder', 'readmitted': 'manual', 'gender': 'labelencoder'}, {'manual': {'age': {'unknown': -1, '[0-10)': 4, '[10-20)': 14, '[20-30)': 24, '[30-40)': 34, '[40-50)': 44, '[50-60)': 54, '[60-70)': 64, '[70-80)': 74, '[80-90)': 84, '[90-100)': 94}, 'admission_type_id': {'unknown': -1, 2: 1, 7: 1, 6: 5, 8: 5, 4: 3}, 'admission_source_id': {'unknown': -1, 2: 1, 3: 1, 5: 4, 6: 4, 22: 4, 10: 7, 25: 7, 17: 9, 20: 9, 13: 11, 14: 11}, 'max_glu_serum': {'unknown': -1, '>300': 1, '>200': 1, 'Norm': 0, 'None': -1}, 'A1Cresult': {'unknown': -1, '>7': 1, '>8': 1, 'Norm': 0, 'None': -1}, 'readmitted': {'unknown': -1, '>30': 1, '<30': 1, 'NO': 0}}, 'labelencoder': {'race': 'dataset_diabetes/r

In [13]:
df.to_csv(dirPath + "dataCategorizedNew.csv", sep=',', header=True)
